In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
from itertools import combinations
import matplotlib.patches as mpatches

import numpy as np
import os
import shutil
import cv2

from tqdm import trange, tqdm

#
#from numba import jit
#from numba.experimental import jitclass
from numba import jit
#from numba import generated_jit, types

# 
from scipy import signal

#    
from scipy.io import loadmat
import scipy

# 
import csv

from Analyze import (plot_pairwise_interactions, 
                     plot_pairwise_interactions2, 
                     ethogram_social_huddling)
                    


Autosaving every 180 seconds


In [3]:
###############################################
###############################################
###############################################

#
fname = '/media/cat/256GB/dan/cohort2/cluster_slp_files/track_all_day_[0, 1, 2, 3, 4, 5].npy'
locs = np.load(fname)#.astype('float64')
print (locs.shape)

# binarize data into grid world of 100 pixels 
bin_size = 100

#
locs_binned = locs//bin_size
print (locs_binned.shape)

#



(1872130, 6, 2)
(1872130, 6, 2)


/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in floor_divide
  


In [4]:
###############################################
########### TRY IDENTITY FREE FIRST ###########
###############################################

@jit
def get_binarized_movies_pooled_mice(locs_binned,
                                     frame_grid):
    
    ''' Here we don't care about mice identity
    '''

    # 
    for k in range(locs_binned.shape[0]):
        #
        for a in range(locs_binned.shape[1]):
            if np.isnan(locs_binned[k][a]).sum()==0:
                x= int(locs_binned[k][a][0])
                y= int(locs_binned[k][a][1])
                frame_grid[k,
                           x, 
                           y] = 1

    return frame_grid

#
# make a 12 x 12 grid to start
frame_grid = np.zeros((locs_binned.shape[0],12,12),dtype='float32')

#
frame_grid = get_binarized_movies_pooled_mice(locs_binned, 
                                  frame_grid)
print ("DONE...")
print ("frame_grid: ", frame_grid.shape)

#################################
plt.figure()
for k in range(100):
    ax=plt.subplot(10,10,k+1)
    idx = np.random.choice(frame_grid.shape[0])
    #idx = k*1000
    plt.imshow(frame_grid[idx]
               )
    plt.xticks([])
    plt.yticks([])
    plt.title("# "+str(idx),
             fontsize=10,
             pad=0.9)
plt.show()


DONE...
frame_grid:  (1872130, 12, 12)


In [5]:
################################################
###################### PCA #####################
################################################
#
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

#
X = frame_grid.reshape(frame_grid.shape[0], -1)
                       
#                       
pca_data = pca.fit_transform(X)
print (pca_data.shape)

#
plt.figure()
plt.scatter(pca_data[::10,0],
            pca_data[::10,1])

plt.show()

(1872130, 2)


In [19]:
##############################################
from sklearn.cluster import Birch
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture

#X= pca_data[::10].astype('float32')

#gm = GaussianMixture(n_components=20, random_state=0).fit(X)

print(gm)
#
clustering = gm.predict(X)


print (clustering)

#
n_clusters = np.unique(clustering)
print ("# clusters: ", n_clusters)

#
clrs = np.zeros(X.shape[0])
for k in n_clusters:
    idx = np.where(clustering==k)[0]
    clrs[idx]=k

#############################################
plt.figure()
plt.scatter(X[:,0],
            X[:,1],
            c=clrs,
            cmap=cm.gist_ncar)

plt.show()





GaussianMixture(n_components=20, random_state=0)
[ 5 19 19 ... 19 19 19]
# clusters:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [ ]:
plt.figure()
for k in range(100):
    ax=plt.subplot(10,10,k+1)
    idx = np.random.choice(frame_grid.shape[0])
    #idx = k*1000
    plt.imshow(frame_grid[idx]
               )
    plt.xticks([])
    plt.yticks([])
    plt.title("# "+str(idx),
             fontsize=10,
             pad=0.9)
plt.show()